In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import urllib

In [3]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [22]:
url='https://www.kohls.com/catalog/sale.jsp?CN=Promotions:Sale&cc=saleclearance-TN1.0-S-saleclearance&kls_sbp=79308538538477351472440984818918846539'

page=urllib.request.urlopen(url).read()

In [23]:
clean=BeautifulSoup(page,"html.parser")

products=[]

productlist=clean.find_all("ul",{"class":"products"})
for content in productlist:
  for prod1 in content.find_all("li",{"class":"products_grid "}):
    products.append(prod1)
  for prod2 in content.find_all("li",{"class":"products_grid yourPrice_eligible"}):
    products.append(prod2)

In [24]:
len(products)

34

In [25]:
products[1]

<li class="products_grid yourPrice_eligible">
<div class="prod_img_block">
<a data-altimg="https://media.kohlsimg.com/is/image/kohls/1695762_ALT" data-prdcount="" href="/product/prd-1695762/gloria-vanderbilt-amanda-tapered-jeans-womens.jsp?color=Rinse Noir&amp;prdPV=3" rel="/product/prd-1695762/gloria-vanderbilt-amanda-tapered-jeans-womens.jsp?color=Rinse Noir&amp;prdPV=3">
<img alt="Women's Gloria Vanderbilt Amanda Classic Jeans" class="pmp-hero-img" data-herosrc="https://media.kohlsimg.com/is/image/kohls/1695762_Rinse_Noir?wid=240&amp;hei=240&amp;op_sharpen=1" src="/snb/media/44.0.0-1581/images/blank_1x1.png" title="Women's Gloria Vanderbilt Amanda Classic Jeans"/>
</a>
</div>
<div class="prod_badgesblock top-rated">
<div class="prod_badge"><img alt="Best Seller" src="/snb/media/44.0.0-1581/images/new_product_badges/BestSeller@2x.png" title="Best Seller"/></div>
</div>
<div class="prod_colorswatches" data-list="2">
<div class="colorSwatch_cont">
<span class="colorSwatch">
<a href="ja

In [27]:
data=[]

for product in products:
  image=product.find_all('img')[0]['data-herosrc']
  name=product.find_all('p')[0].text.strip()
  deal_price=product.find_all('span',{'class':'prod_price_amount red_color'})[0].text
  original_price=product.find_all('div',{'class':'prod_price_original'})[0].text
  link='https://www.kohls.com/' + product.find_all('a')[0]['href']

  row=pd.Series({'Image':image,'Name':name,'Link':link,'Deal_Price':deal_price, 'Original_Price':original_price})

  data.append(row)

data=pd.concat(data,axis=1).T

In [28]:
def make_clickable(val):
    return f'<a target="_blank" href="{val}">{val}</a>'

data=data.style.format({'Link': make_clickable,'Image':make_clickable})

In [29]:
data

,Image,Name,Link,Deal_Price,Original_Price
0,https://media.kohlsimg.com/is/image/kohls/2861368_Shadow_Gray?wid=240&hei=240&op_sharpen=1,Simply Vera Vera Wang Supima Cotton 600 Thread Count Deep Pocket Sheet Set with Pillowcases,https://www.kohls.com//product/prd-2861368/simply-vera-vera-wang-4-piece-600-thread-count-sheet-set.jsp?color=Shadow Gray&prdPV=2,$39.99 - $119.99,Reg. $49.99 - $149.99
1,https://media.kohlsimg.com/is/image/kohls/1695762_Rinse_Noir?wid=240&hei=240&op_sharpen=1,Women's Gloria Vanderbilt Amanda Classic Jeans,https://www.kohls.com//product/prd-1695762/gloria-vanderbilt-amanda-tapered-jeans-womens.jsp?color=Rinse Noir&prdPV=3,$16.00 - $24.99,Reg. $40.00
2,https://media.kohlsimg.com/is/image/kohls/3404073_Slate_Blue?wid=240&hei=240&op_sharpen=1,Simply Vera Vera Wang 800 Thread Count Egyptian Cotton Sheet Set or Pillowcases,https://www.kohls.com//product/prd-3404073/simply-vera-vera-wang-800-thread-count-sheet-set.jsp?color=Slate Blue&prdPV=4,$64.99 - $159.99,Reg. $89.99 - $199.99
3,https://media.kohlsimg.com/is/image/kohls/1944597_Dogs_With_Bones?wid=240&hei=240&op_sharpen=1,The Big One® Oversized Supersoft Plush Throw,https://www.kohls.com//product/prd-1944597/the-big-one-super-soft-plush-throw.jsp?color=Dogs With Bones&prdPV=6,$24.99,Original $26.99
4,https://media.kohlsimg.com/is/image/kohls/4942292_Warm_Ginger?wid=240&hei=240&op_sharpen=1,Women's Sonoma Goods For Life® Favorite Long Sleeve Cardigan,https://www.kohls.com//product/prd-4942292/womens-sonoma-goods-for-life-favorite-long-sleeve-cardigan.jsp?color=Warm Ginger&prdPV=7,$29.99,Original $36.00
5,https://media.kohlsimg.com/is/image/kohls/2954787_Linden?wid=240&hei=240&op_sharpen=1,Women's Sonoma Goods For Life® Supersoft Stretch Midrise Skinny Jeans,https://www.kohls.com//product/prd-2954787/womens-sonoma-goods-for-life-faded-skinny-jeans.jsp?color=Linden&prdPV=8,$24.99,Reg. $36.00
6,https://media.kohlsimg.com/is/image/kohls/4569317_Gray?wid=240&hei=240&op_sharpen=1,Sonoma Goods For Life® Ultimate Bath Towel with Hygro® Technology,https://www.kohls.com//product/prd-4569317/sonoma-goods-for-life-ultimate-towel-with-hygro-technology.jsp?color=Gray&prdPV=9,$5.99 - $14.99,Reg. $7.99 - $19.99
7,https://media.kohlsimg.com/is/image/kohls/3836990_Ocean_Mist?wid=240&hei=240&op_sharpen=1,Women's Croft & Barrow® Essential Long Sleeve Crewneck Tee,https://www.kohls.com//product/prd-3836990/womens-croft-barrow-classic-crewneck-tee.jsp?color=Ocean Mist&prdPV=10,$13.99,Original $16.00
8,https://media.kohlsimg.com/is/image/kohls/4569328_Medium_Gray?wid=240&hei=240&op_sharpen=1,The Big One® Solid Bath Towel,https://www.kohls.com//product/prd-4569328/the-big-one-solid-towel.jsp?color=Medium Gray&prdPV=11,$2.99 - $4.99,Reg. $3.99 - $6.99
9,https://media.kohlsimg.com/is/image/kohls/3292246_Graphite?wid=240&hei=240&op_sharpen=1,Keurig® K-Select® Single-Serve K-Cup Pod® Coffee Maker with Strength Control,https://www.kohls.com//product/prd-3292246/keurig-k-select-single-serve-k-cup-pod-coffee-maker.jsp?color=Graphite&prdPV=12,$139.99,Reg. $149.99


In [8]:
data.to_excel('khols.xlsx',index=False)